# BI output results and speed comparaison
## 1. Continuous variable
### 1.1. Speed comparaison

In [1]:
import os
import sys
newPath = os.path.dirname(os.path.abspath(""))
if newPath not in sys.path:
    sys.path.append(newPath)
from src.main import*
import time as tm

2024-03-15 14:39:55.645848: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 14:39:55.666791: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 14:39:55.666810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 14:39:55.667412: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 14:39:55.671287: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import time as tm
from src.main import*
## Model m4.3
d = pd.read_csv('../data/Howell1.csv', sep=';')
d = d[d.age > 18]
#self.df["weight.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
d.weight = d.weight - d.weight.mean()
d.age = d.age - d.age.mean()
formula = dict(main1 = 'height ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha + beta * weight',
            prior1 = 'sigma ~ Uniform(0,50)',
            prior2 = 'alpha ~ Normal(178,20)',
            prior3 = 'beta ~ Normal(0,1)')    

m4_3 = model(formula, df = d, float = 32)
start = tm.time()
m4_3.fit(observed_data = dict(height =d.height.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)   
end = tm.time()    
m4_3.summary(round_to = 'none')     
print(f"BI took: {end - start:.4f} seconds")

2024-03-15 14:39:57.171548: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:39:57.187885: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:39:57.187918: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:39:57.384158: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:39:57.384199: I external/local_xla/xla/stream_executor

BI took: 4.8124 seconds


In [3]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
nest_asyncio.apply()
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass
stan_code = """
data{
  vector[346] height;
  vector[346] weight;
}
parameters{
  real a;
  real<lower=0> b;
  real<lower=0,upper=50> sigma;
}
model{
  vector[346] mu;
  sigma ~ uniform( 0 , 50 );
  b ~ lognormal( 0 , 1 );
  a ~ normal( 178 , 20 );
  for ( i in 1:346 ) {
    mu[i] = a + b* weight[i] ;
  }
  height ~ normal( mu , sigma );  
}
"""
data = {
  'height': d.height.values,
  'weight': d.weight.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 15, column 20: Argument
    20 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 15, column 14: Argument
    178 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_bzl01w9e/model_qxjp4bwu.stan', line 13, column 23: Argument
    50 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    in its declaration. These hard constraints are not recommended, for two
    reasons: (a) Except when there are logical or physical constraints, it is
    very unusual for you to be sure that a parameter will fall inside a
    specified range, and (b) The infinite gradient induced b

Pystan took: 0.9014 seconds


### 1.2. Output comparaison 

In [4]:
pd.DataFrame(
    {
        "tfp": m4_3.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df.b.mean(), df.a.mean()]
    })

,tfp,pystan
sigma[0],1.0,5.144955
beta[0],1.0,0.905427
alpha[0],1.0,154.648967


## 2. Categorical variable
### 2.1. Speed comparaisons

In [5]:
from src.main import*
m5_9 = model()
m5_9.import_csv('../data/milk.csv', sep = ';')
m5_9.df["K"] = m5_9.df["kcal.per.g"].pipe(lambda x: (x - x.mean()) / x.std())
m5_9.index(cols = "clade")
formula = dict(main = 'K ~ Normal(mu,sigma)',
            likelihood = 'mu ~ alpha[index_clade]',
            prior1 = 'alpha~ Normal(0,0.5)',
            prior2 = 'sigma ~ Exponential(1)') 

m5_9.f = formula

start = tm.time()
m5_9.build_model()
m5_9.fit(observed_data = dict(K = m5_9.df.K.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-15 14:40:03.395850: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:40:03.395930: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:40:03.395943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:40:03.396138: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:40:03.396156: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.2565 seconds


In [7]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code = """
data{
    vector[29] K;
    array[29] int clade_id;
}
parameters{
    vector[4] a;
    real<lower=0> sigma;
}
model{
    vector[29] mu;
    sigma ~ exponential( 1 );
    a ~ normal( 0 , 0.5 );
    for ( i in 1:29 ) {
        mu[i] = a[clade_id[i]];
    }
    K ~ normal( mu , sigma );
    
}
"""
data = {
  'clade_id': m5_9.df.index_clade.values+1,
  'K': m5_9.df.K.values,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.4s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.2 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.21 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.2 seconds.
  Adjust your expectations accordingly!


Pystan took: 11.7740 seconds


### 2.2. Output comparaison

In [8]:
pd.DataFrame(
    {
        "tfp": m5_9.summary(round_to = 'none')['mean'],
        "pystan": [df.sigma.mean(),  df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean()]
    })

,tfp,pystan
sigma[0],0.806870,0.799935
alpha[0],-0.466500,-0.470920
alpha[1],0.349950,0.350620
alpha[2],0.634007,0.640167
alpha[3],-0.547865,-0.552527


## 3. Continuous interactions terms (model 8.3)
### 3.1. Speed comparaisons

In [9]:
from src.main import*
d = pd.read_csv('../data/tulips.csv', sep = ';')
d["blooms_std"] = d.blooms / d.blooms.max()
d["water_cent"] = d.water - d.water.mean()
d["shade_cent"] = d.shade - d.shade.mean()

formula = dict(
            main = 'blooms_std ~ Normal( mu , sigma ) ',
            likelihood ='mu ~ a + bw*water_cent + bs*shade_cent + bws*water_cent*shade_cent' ,
            prior1 = 'a ~ Normal( 0.5 , 0.25 ) ',
            prior2 = 'bw ~ Normal( 0 , 0.25 ) ',
            prior3 = 'bs ~ Normal( 0 , 0.25 ) ',
            prior4 = 'bws ~ Normal( 0 , 0.25 ) ',
            prior5 = 'sigma ~ Exponential( 1 )',
            )
start = tm.time()
m8_5 = model(formula, d)
m8_5.fit(observed_data = dict(blooms_std =d.blooms_std.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-15 14:41:03.797274: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:03.797334: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:03.797347: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:03.797456: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:03.797462: I tensorflow/core/common_runtime/gpu/gpu

BI took: 7.3743 seconds


In [10]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """
data{
    vector[27] blooms_std;
    array[27] int shade_cent;
    array[27] int water_cent;
}
parameters{
    real a;
    real bw;
    real bs;
    real bws;
    real<lower=0> sigma;
}
model{
    vector[27] mu;
    sigma ~ exponential( 1 );
    bws ~ normal( 0 , 0.25 );
    bs ~ normal( 0 , 0.25 );
    bw ~ normal( 0 , 0.25 );
    a ~ normal( 0.5 , 0.25 );
    for ( i in 1:27 ) {
        mu[i] = a + bw * water_cent[i] + bs * shade_cent[i] + bws * water_cent[i] * shade_cent[i];
    }

    
    blooms_std ~ normal( mu , sigma );
}
"""

data = {
    'blooms_std' : d["blooms_std"].values,
    "water_cent": d["water_cent"].values.astype(int),
    "shade_cent": d["shade_cent"].values.astype(int),
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.16 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_ti9iwirl/model_epcjb2i5.stan', line 26, column 4 to column 38)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often 

Pystan took: 0.4304 seconds


### 3.2. Output comparaison

In [11]:
pd.DataFrame(
    {
        "tfp": m8_5.summary(round_to='none')['mean'],
        "pystan": [df.sigma.mean(),  df['bws'].mean(), df['bw'].mean(), df['bs'].mean(), df['a'].mean()]
    })

,tfp,pystan
sigma[0],0.142830,0.142646
bws[0],-0.141900,-0.142218
bw[0],0.207684,0.206281
bs[0],-0.113561,-0.112316
a[0],0.358012,0.358629


## 4. Binomial model
### 4.1. Speed comparaisons

In [12]:
from src.main import*
d = pd.read_csv('../data/chimpanzees.csv', sep = ';')
d["treatment"] = 1 + d.prosoc_left + 2 * d.condition
d["side"] = d.prosoc_left  # right 0, left 1
d["cond"] = d.condition  # no partner 0, partner 1

d_aggregated = (
    d.groupby(["treatment", "actor", "side", "cond"])["pulled_left"].sum().reset_index()
)
d_aggregated.rename(columns={"pulled_left": "left_pulls"}, inplace=True)
d_aggregated["actor_id"] = d_aggregated["actor"].values - 1

formula = dict(
    main = 'pulled_left ~ Binomial( 1 , logits = p )' ,
    likelihood = 'p ~ a' ,
    prior1 = 'a ~ Normal( 0 , 10 )'
)
start = tm.time()
m11_1 = model(formula, d)
m11_1.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-15 14:41:11.637160: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:11.637237: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:11.637254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:11.637427: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:11.637440: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.6867 seconds


In [13]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
}
parameters{
    real a;
}
model{
    real p;
    a ~ normal( 0 , 10 );
    p = a;
    p = inv_logit(p);
    pulled_left ~ binomial( 1 , p );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.03 seconds.
  Adjust your expectations accordingly!


Pystan took: 0.3301 seconds


### 4.2. Output comparaison

In [14]:
pd.DataFrame(
    {
        "tfp": m11_1.summary(round_to='none')['mean'],
        "pystan": [df.a.mean()]
    })

,tfp,pystan
a[0],0.32029,0.319077


##  5. Binomial with indices
### 5.1.Speed comparaisons

In [15]:
from src.main import*
d = pd.read_csv('../data/chimpanzees.csv', sep = ';')
d.actor = d.actor - 1
d["treatment"] = d.prosoc_left + 2 * d.condition
d[["actor", "prosoc_left", "condition", "treatment"]]

formula = dict(
    main = 'pulled_left ~ Binomial(1 , p )' ,
    likelihood = 'p ~ a[actor] + b[treatment]' ,
    prior1 = 'a ~ Normal(0, 1.5)',
    prior2 = 'b ~ Normal(0, 0.5)'
)

start = tm.time()
m11_4 = model(formula, d, float = 32)
m11_4.fit(observed_data = dict(pulled_left =d.pulled_left.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")


2024-03-15 14:41:16.691780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:16.691845: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:16.691859: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:16.692116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:16.692127: I tensorflow/core/common_runtime/gpu/gpu

BI took: 6.4688 seconds


In [16]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """
data{
    array[504] int pulled_left;
    array[504] int treatment;
    array[504] int actor;
}
parameters{
    vector[7] a;
    vector[4] b;
}
model{
    vector[504] p;
    b ~ normal( 0 , 0.5 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    pulled_left ~ binomial( 1 , p );
}
generated quantities{
    vector[504] log_lik;
    vector[504] p;
    for ( i in 1:504 ) {
        p[i] = a[actor[i]] + b[treatment[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:504 ) log_lik[i] = binomial_lpmf( pulled_left[i] | 1 , p[i] );
    
}
"""

data = {
    'pulled_left' : d["pulled_left"].values.astype(int),
    'treatment' : d["treatment"].values.astype(int) +1,
    'actor' : d["actor"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 3.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.37 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.37 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.35 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.39 seconds.
  Adjust your expectations accordingly!


Pystan took: 3.6397 seconds


### 5.2. Output comparaison

In [17]:
pd.DataFrame(
    {
        "tfp": m11_4.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), df['b.3'].mean(), df['b.4'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(), df['a.3'].mean(), df['a.4'].mean(), df['a.5'].mean(), df['a.6'].mean(), df['a.7'].mean()]
    })

,tfp,pystan
b[0],-0.033493,-0.031740
b[1],0.479430,0.488475
b[2],-0.383442,-0.378498
b[3],0.372759,0.377438
a[0],-0.448006,-0.460654
a[1],3.897117,3.879655
a[2],-0.748344,-0.757164
a[3],-0.749810,-0.754823
a[4],-0.453015,-0.454010
a[5],0.473095,0.470781


## 6. Poisson
### 6.1. Speed comparaison

In [18]:
from src.main import*
d = pd.read_csv('../data/Kline.csv', sep = ';')
d["P"] = d.population.pipe(np.log).pipe(lambda x: (x - x.mean()) / x.std())
d["cid"] = (d.contact == "high").astype(int)
d['pLog'] = tf.math.log(d.P).numpy()
formula = dict(main = 'total_tools ~ Poisson(log_rate = lambda)',
               likelihood = 'lambda ~ alpha[cid] + beta[cid]*P',
               prior1 = 'alpha ~ Normal(3,0.5)',
               prior2 = 'beta ~ Normal(0,0.2)')
start = tm.time()
m11_10 = model(formula, d)
m11_10.fit(observed_data = dict(total_tools =d.total_tools.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-15 14:41:26.927491: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:26.927562: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:26.927576: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:26.928109: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:41:26.928122: I tensorflow/core/common_runtime/gpu/gpu

BI took: 5.2638 seconds


In [19]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code = """ 
data{
    array[10] int T;
    vector[10] P;
    array[10] int cid;
}
parameters{
    vector[2] a;
    vector[2] b;
}
model{
    vector[10] lambda;
    b ~ normal( 0 , 0.2 );
    a ~ normal( 3 , 0.5 );
    for ( i in 1:10 ) {
       lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
       lambda[i] = exp(lambda[i]);
    }
    T ~ poisson( lambda );
}
generated quantities{
    vector[10] log_lik;
    vector[10] lambda;
    for ( i in 1:10 ) {
        lambda[i] = a[cid[i]] + b[cid[i]] * P[i];
        lambda[i] = exp(lambda[i]);
    }
    for ( i in 1:10 ) log_lik[i] = poisson_lpmf( T[i] | lambda[i]);
    
}
"""

data = {
    'T' : d["total_tools"].values.astype(int),
    'P' : d["P"].values.astype(float),
    'cid' : d["cid"].values.astype(int) +1
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.1 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.06 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.05 seconds.
  Adjust your expectations accordingly!


Pystan took: 0.5402 seconds


### 6.2. Output comparaison 

In [20]:
pd.DataFrame(
    {
        "tfp": m11_10.summary(round_to='none')['mean'],
        "pystan": [df['b.1'].mean(), df['b.2'].mean(), 
                   df['a.1'].mean(), df['a.2'].mean()]
    })

,tfp,pystan
beta[0],0.378112,0.376990
beta[1],0.192942,0.192069
alpha[0],3.320171,3.320298
alpha[1],3.610382,3.610055


## 7. Negative binomial
### 7.1. Speed comparaison

In [107]:
from src.main import*
num_days = 30
y = tfd.Poisson(rate=1.5).sample((num_days,))
num_weeks = 4
y_new = tfd.Poisson(rate=0.5 * 7).sample((num_weeks,))
y_all = np.concatenate([y, y_new])
exposure = np.concatenate([np.repeat(1, 30), np.repeat(7, 4)])
monastery = np.concatenate([np.repeat(0, 30), np.repeat(1, 4)])
d = pd.DataFrame.from_dict(dict(y=y_all, days=exposure, monastery=monastery))
d["log_days"] = d.days.pipe(np.log)

formula = dict(main = 'y ~ Poisson(rate = lambda)',
               likelihood = 'lambda ~ tf.exp(log_days + alpha +  beta * monastery)',
               prior1 = 'alpha ~ Normal(0,1)',
               prior2 = 'beta ~ Normal(0,1)')
start = tm.time()
m11_12 = model(formula, d, float=32)
m11_12.fit(observed_data = dict(y =d.y.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

2024-03-15 15:37:02.333055: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 15:37:02.333138: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 15:37:02.333154: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 15:37:02.333261: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 15:37:02.333267: I tensorflow/core/common_runtime/gpu/gpu

BI took: 7.6505 seconds


In [129]:
m = {}
m['a'] = tfd.Sample(tfd.Normal(loc=0.0, scale=1.0), sample_shape=1)
m['b'] = tfd.Sample(tfd.Normal(loc=0.0, scale=1.0), sample_shape=1)
m['y'] = lambda a, b:  tfd.Independent(
            tfd.Poisson(rate=tf.exp(d.log_days.values + a[..., tf.newaxis] + b[..., tf.newaxis] * d.monastery.values)), reinterpreted_batch_ndims=1
        )
m = tfd.JointDistributionNamedAutoBatched(m)
m.sample()

{'b': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.03662567], dtype=float32)>,
 'a': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-0.16287757], dtype=float32)>,
 'y': <tf.Tensor: shape=(1, 34), dtype=float32, numpy=
 array([[ 0.,  1.,  1.,  0.,  2.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
          1.,  2.,  0.,  2.,  2.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,
          0.,  1.,  1.,  0.,  9., 10.,  4.,  5.]], dtype=float32)>}

In [139]:
m11_12.tensor = m
m11_12.fit(observed_data = dict(y =d.y.astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)


In [140]:
m11_12.summary()

,mean,sd,hdi_5.5%,hdi_94.5%
b[0],-1.10,0.30,-1.61,-0.66
a[0],0.38,0.14,0.15,0.59


In [28]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code =""" 
data{
    array[34] int days;
    array[34] int y;
    array[34] int monastery;
    vector[34] log_days;
}
parameters{
    real a;
    real b;
}
model{
    vector[34] lambda;
    b ~ normal( 0 , 1 );
    a ~ normal( 0 , 1 );
    for ( i in 1:34 ) {
        lambda[i] = log_days[i] + a + b * monastery[i];
        lambda[i] = exp(lambda[i]);
    }
    
    y ~ poisson( lambda );
    
}
"""

data = {
    'days' : d["days"].values.astype(int),
    'y' : d["y"].values.astype(int),
    'monastery' : d["monastery"].values.astype(int) +1,
    'log_days' : d["log_days"].values.astype(float),
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 12.9s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.15 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 9e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.09 seconds.
  Adjust your expectations accordingly!


Pystan took: 13.4252 seconds


### Output comparaison (PB estimation)

In [108]:
pd.DataFrame(
    {
        "tfp": m11_12.summary(round_to='none')['mean'],
        "pystan": [df['b'].mean(), df['a'].mean()]
    })

,tfp,pystan
beta[0],-1.099935,-0.983513
alpha[0],0.383595,1.322167


## 8. Multinomial
TODO : This model can't account for indices in the formula nor varying intercepts of varying effects.
### 8.1. Speed comparaison

In [ ]:
# simulate career choices among 500 individuals
N = 500  # number of individuals
income = np.array([1, 2, 5])  # expected income of each career
score = 0.5 * income  # scores for each career, based on income

# next line converts scores to probabilities
p = tf.nn.softmax(score)

# now simulate choice
# outcome career holds event type values, not counts
career = np.repeat(np.nan, N)  # empty vector of choices for each individual

# sample chosen career for each individual
for i in range(N):
    career[i] = tfd.Categorical(probs=p).sample()

career = career.astype(int)
result = [income[index] for index in career]
data = {'career': career, 'income': result}
d = pd.DataFrame(data)

2024-03-15 13:54:29.938395: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 13:54:29.954373: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 13:54:29.954407: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 13:54:29.957346: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 13:54:29.957379: I external/local_xla/xla/stream_executor

In [ ]:
formula = dict(main = 'y ~ Poisson(log_rate = p)',
               likelihood = 'p ~ alpha[income] + beta * income',
               prior1 = 'alpha ~ Normal(0,1)',
               prior2 = 'beta ~ Normal(0, 0.5)')
m11_13 = model(formula, d)
m11_13.prior_dict

2024-03-15 14:32:43.869962: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870024: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870038: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:32:43.870231: I tensorflow/core/common_runtime/gpu/gpu

{'alpha': "tfd.Sample(tfd.Normal(0,1, name = 'prior1'), sample_shape = 3)",
 'beta': "tfd.Sample(tfd.Normal(0,0.5, name = 'prior2'), sample_shape = 1)"}

In [ ]:
m11_13.main_dict

{'y': "lambda alpha, beta : tfd.Independent(tfd.Poisson(log_rate= tf.squeeze(tf.gather(alpha,tf.cast(df.income.astype('float32').values, dtype=tf.int32), axis = -1))+beta*df.income.astype('float32').values, name ='main'), reinterpreted_batch_ndims=1)"}

In [ ]:
formula = dict(main = 'y ~ Categorical(probs = p, cat = income)',
               likelihood = 'p ~ alpha[income] + beta * income',
               prior1 = 'alpha ~ Normal(0, 1)',
               prior2 = 'beta ~ HalfNormal(0.5)'
               )
start = tm.time()   
m11_13 = model(formula, d)
m11_13.fit(observed_data = dict(y =d.career.astype('float32').values), num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

tf.nn.softmax(tf.stack([alpha[0]+beta*1.0,alpha[1]+beta*2.0,0 +beta*2.0], axis=1))


2024-03-15 14:16:45.724321: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:16:45.724371: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:16:45.724384: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:16:45.724494: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 14:16:45.724499: I tensorflow/core/common_runtime/gpu/gpu

BI took: 8.1127 seconds


In [ ]:
import pymc as pm
with pm.Model() as m11_13_pm:
    a = pm.Normal("a", 0.0, 1.0, shape=2)  # intercepts
    b = pm.HalfNormal("b", 0.5)  # association of income with choice

    s0 = a[0] + b * income[0]
    s1 = a[1] + b * income[1]
    s2 = 0.0 + b * income[2]  # pivoting the intercept for the third category
    s = pm.math.stack([s0, s1, s2])

    p_ = pm.math.softmax(s)
    career_obs = pm.Categorical("career", p=p_, observed=career)

    trace_11_13 = pm.sample(tune=2000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b]


Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 8 seconds.


In [ ]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()

stan_code =""" 
data{
    int N; // number of individuals
    int K; // number of possible careers
    array[N] int career; // outcome
    vector[K] career_income;
}
parameters{
    vector[K-1] a; // intercepts
    real<lower=0> b; // association of income with choice
}
model{
    vector[K] p;
    vector[K] s;
    a ~ normal( 0 , 1 );
    b ~ normal( 0 , 0.5 );
    s[1] = a[1] + b*career_income[1];
    s[2] = a[2] + b*career_income[2];
    s[3] = 0; // pivot
    p = softmax( s );
    career ~ categorical( p );
}
"""

data = {
    'N' : 500,
    'K' : 3,
    'career' : d["career"].values.astype(int) + 1,
    'career_income' : d["income"].unique().astype(int).tolist(),
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 12.5s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 2.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.23 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.26 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.23 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 2.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.24 seconds.
  Adjust your expectations accordingly!


Pystan took: 12.9668 seconds


In [ ]:
tmp = az.summary(trace_11_13, round_to=2)['mean']
pd.DataFrame(
    {
        "tfp": m11_13.summary(round_to='none')['mean'],
        "pystan": [df['b'].mean(), df['a.1'].mean(), df['a.2'].mean()],
        "pymc": [tmp[2], tmp[0], tmp[1]]
    })
# It seems that beta should be 0.5

/tmp/ipykernel_187862/960065487.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "pymc": [tmp[2], tmp[0], tmp[1]]


,tfp,pystan,pymc
beta[0],0.529352,0.056154,0.43
alpha[0],-1.439973,-2.218914,-0.29
alpha[1],-1.233069,-1.342226,0.03


## 9. Beta binomial
### 9.1. Speed comparaison

In [186]:
from src.main import*
d = pd.read_csv('../data/UCBadmit.csv', sep = ';')
d["gid"] = (d["applicant.gender"] != "male").astype(int)
len(d.applications)
formula = dict(main = 'y ~ BetaBinomial(applications, concentration1 = pbar*theta, concentration0 = (1 - pbar) * theta)',
               likelihood = 'pbar ~ sigmoid(alpha[gid])',
               likelihood2 = 'theta ~ phi + 2.0',
               prior1 = 'alpha ~ Normal(0.,1.5)',
               prior2 = 'phi ~ Exponential(1)'
               )
start = tm.time()
m12_1 = model(formula, d)
m12_1.fit(observed_data = dict(y =d.admit.astype('float32').values),
                                           num_results = 1000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

2024-03-15 16:28:03.249672: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 16:28:03.249737: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 16:28:03.249753: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 16:28:03.249917: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-15 16:28:03.249922: I tensorflow/core/common_runtime/gpu/gpu

BI took: 6.2144 seconds


In [183]:
m12_1.main_dict

{'y': "lambda alpha, phi : tfd.Independent(tfd.BetaBinomial(12,concentration1=tf.sigmoid( tf.squeeze(tf.gather(alpha,tf.cast(df.gid.astype('float32').values, dtype=tf.int32), axis = -1)))*phi+2.0,concentration0=(1-tf.sigmoid( tf.squeeze(tf.gather(alpha,tf.cast(df.gid.astype('float32').values, dtype=tf.int32), axis = -1))))*phi+2.0, name ='main'), reinterpreted_batch_ndims=1)"}

In [185]:
len(d["applications"].values.astype(int))

12

In [181]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code =""" 
data{
    array[12] int N;
    array[12] int A;
    array[12] int gid;
}
parameters{
    vector[2] a;
    real<lower=0> phi;
}
transformed parameters{
    real theta;
    theta = phi + 2;
}
model{
    vector[12] pbar;
    phi ~ exponential( 1 );
    a ~ normal( 0 , 1.5 );
    for ( i in 1:12 ) {
        pbar[i] = a[gid[i]];
        pbar[i] = inv_logit(pbar[i]);
    }
    A ~ beta_binomial( N , pbar*theta , (1-pbar)*theta );    
}
"""

data = {
    'A' : d["admit"].values.astype(int),
    'N' : d["applications"].values.astype(int),
    'gid' : d["gid"].values.astype(int) +1,
}

start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: found in cache, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 8e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.08 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: beta_binomial_lpmf: Second prior sample size parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_r_32hiex/model_aokd6ccs.stan', line 23, column 4 to column 57)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 8e-06 seconds
  1000 transitions 

Pystan took: 0.4592 seconds


### 9.2. Output comparaison (PB estimation)

In [187]:
pd.DataFrame(
    {
        "tfp": m12_1.summary(round_to='none')['mean'],
        "pystan": [df['phi'].mean(), df['a.1'].mean(), df['a.2'].mean()]
    })

,tfp,pystan
phi[0],0.650548,1.030113
alpha[0],-0.676216,-0.442224
alpha[1],-0.513588,-0.312028


## 10. Negative-binomial
### 10.1. Speed comparaison

In [ ]:
from src.main import*
d = pd.read_csv('../data/Kline).csv', sep = ';')
formula = dict(main = 'y ~ BetaBinomial(12,  concentration1 = pbar/phi, concentration0 = g_rate)',
               likelihood1 = 'pbar ~ tf.exp(alpha[gid])*tf.math.pow(beta[gid])/gamma',
               likelihood2 = 'g_rate ~ 1/phi',
               prior1 = 'alpha ~ Normal(1,1)',
               prior2 = 'beta ~ Exponential(1)',
               prior3 = 'gamma ~ Exponential(1)',
               prior4 = 'phi ~ Exponential(1)',
               )
start = tm.time()
m12_1 = model(formula, d)
m12_1.main_dict

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp

tf.random.set_seed(42)

# Define the model
def model():
    m = {}
    m['alpha'] = tfp.distributions.Sample(tfp.distributions.Normal(1, 1), sample_shape=2)
    m['beta'] = tfp.distributions.Sample(tfp.distributions.Exponential(1), sample_shape=2)
    m['gamma'] = tfp.distributions.Sample(tfp.distributions.Exponential(1), sample_shape=1)
    m['phi'] = tfp.distributions.Sample(tfp.distributions.Exponential(1), sample_shape=1)
    
    # Define the distribution 'y' which depends on alpha, beta, gamma, and phi
    def y_dist(phi, alpha, beta, gamma):
        concentration1 = ((tf.exp(alpha) * beta) / gamma) / phi
        concentration0 = 1. / phi
        return tfp.distributions.Independent(
            tfp.distributions.BetaBinomial(12, concentration1=concentration1, concentration0=concentration0),
            reinterpreted_batch_ndims=1)

    m['y'] = y_dist
    return tfp.distributions.JointDistributionNamedAutoBatched(m)

# Sample from the model
sampled_values = model().sample(alpha=tf.random.normal((2,)), beta=tfp.distributions.Exponential(rate=1.).sample((2,)), gamma=tfp.distributions.Exponential(rate=1.).sample((1,)), phi=tfp.distributions.Exponential(rate=1.).sample((1,)))

print(sampled_values)



In [ ]:
df = d
m = {}
m['alpha']= tfd.Sample(tfd.Normal(1,1, name = 'prior1'), sample_shape = 2)
m['beta']= tfd.Sample(tfd.Exponential(1, name = 'prior2'), sample_shape = 2)
m['gamma']= tfd.Sample(tfd.Exponential(1, name = 'prior3'), sample_shape = 1)
m['phi']= tfd.Sample(tfd.Exponential(1, name = 'prior4'), sample_shape = 1)
m['y'] = lambda phi, alpha, beta, gamma : tfd.Independent(
    tfd.BetaBinomial(12,
                     concentration1=(
                         tf.exp(tf.squeeze(tf.gather(alpha,tf.cast(df.gid.astype('float32').values, dtype=tf.int32), axis = -1)))
                         *( tf.math.powtf.squeeze(tf.gather(beta,tf.cast(df.gid.astype('float32').values, dtype=tf.int32), axis = -1)))
                         /gamma)/phi,
                     concentration0=(1/phi), name ='main'), reinterpreted_batch_ndims=1)
m = tfd.JointDistributionNamedAutoBatched(m)
m.sample()

## 11. Zero inflated outcomes
### 11.1. Speed comparaison

In [ ]:
from src.main import*
import random
random.seed(42)
# Define parameters
prob_drink = 0.2  # 20% of days
rate_work = 1     # average 1 manuscript per day

# sample one year of production
N = 365

np.random.seed(365)
drink = np.random.binomial(1, prob_drink, N)
y = (1 - drink) * np.random.poisson(rate_work, N)
d = pd.DataFrame(y)
formula = dict(main = 'y ~ ZeroInflatedNegativeBinomial(total_count = 365, inflated_loc_logits = p, logits = AL)',
               likelihood = "p ~ ap",
               likelihood2 = "AL ~ tf.math.log(al)",
               prior1 = 'ap ~ Normal(-1.5 , 1)',
               prior2 = 'al ~ Normal(1,0.5)'
               )

start = tm.time()
m12_3 = model()
m12_3 = model(formula)       
m12_3.fit(observed_data = dict(y = d.iloc[:,0].astype('float32').values),
                                           num_results = 2000, num_burnin_steps=500, num_adaptation_steps=400, num_chains=4)
end = tm.time()
print(f"BI took: {end - start:.4f} seconds")

2024-03-12 08:56:24.978128: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978175: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978188: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978344: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-12 08:56:24.978355: I tensorflow/core/common_runtime/gpu/gpu

BI took: 4.8690 seconds


In [ ]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code = """ 
data{
    array[365] int y;
}
parameters{
    real ap;
    real al;
}
model{
    real p;
    real lambda;
    al ~ normal( 1 , 0.5 );
    ap ~ normal( -1.5 , 1 );
    lambda = al;
    lambda = exp(lambda);
    p = ap;
    p = inv_logit(p);
    for ( i in 1:365 ) {
        if ( y[i]==0 )
            target += log_mix( p , 0 , poisson_lpmf(0|lambda) );
        if ( y[i] > 0 )
            target += log1m( p ) + poisson_lpmf(y[i] | lambda );
    }
}
"""
data = {
    'y' :d.iloc[:,0].values.astype(int)
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 9.8s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 5.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.52 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.52 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.71 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.55 seconds.
  Adjust your expectations accordingly!


Pystan took: 11.2434 seconds


### 11.2. Output comparaison (PB estimation)

In [ ]:
pd.DataFrame(
    {
        "tfp": m12_3.summary(round_to='none')['mean'],
        "pystan": [df['ap'].mean(), df['al'].mean()]
    })

,tfp,pystan
ap[0],-0.687577,-1.352273
al[0],0.381126,0.111060


## 12. Varying interceps
### 12.1. Speed comparaison

In [ ]:
from src.main import*
import time as tm
d = pd.read_csv('../data/reedfrogs.csv', sep = ';')
d["tank"] = np.arange(d.shape[0])
formula = dict(main = 'y ~ Binomial(total_count = density, logits = p)',
               likelihood = 'p ~ alpha[tank]', 
               prior = 'alpha ~ Normal(a_bar, sigma)',
               prior1 = 'a_bar ~ Normal(0.,1.5)',
               prior2 = 'sigma ~ Exponential(1)'
               )

start = tm.time()   
m13_2 = model(formula, d, float=32)
m13_2.fit(observed_data = dict(y =d.surv.astype('float32').values), num_chains= 4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-14 10:07:57.845488: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:07:57.845545: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:07:57.845561: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:07:57.845722: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:07:57.845734: I tensorflow/core/common_runtime/gpu/gpu

BI took: 6.0197 seconds


In [ ]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code = """ 
data{
    array[48] int N;
    array[48] int S;
    array[48] int tank;
}
parameters{
    vector[48] a;
    real a_bar;
    real<lower=0> sigma;
}
model{
    vector[48] p;
    sigma ~ exponential( 1 );
    a_bar ~ normal( 0 , 1.5 );
    a ~ normal( a_bar , sigma );
    for ( i in 1:48 ) {
        p[i] = a[tank[i]];
        p[i] = inv_logit(p[i]);
    }
    S ~ binomial( N , p );
}
generated quantities{
    vector[48] log_lik;
    vector[48] p;
    for ( i in 1:48 ) {
        p[i] = a[tank[i]];
        p[i] = inv_logit(p[i]);
    }
    for ( i in 1:48 ) log_lik[i] = binomial_lpmf( S[i] | N[i] , p[i] );    
}
"""
data = {
    'S' : d['surv'].values.astype(int),
    'N' : d['density'].values.astype(int),
    'tank' : d['tank'].values.astype(int)+1,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 11.6s, done.Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: normal_lpdf: Scale parameter is 0, but must be positive! (in '/tmp/httpstan_th_htuzh/model_nkw3miai.stan', line 16, column 4 to column 32)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition wo

Pystan took: 12.9339 seconds


### 12.3 Output comparaison

In [ ]:
pd.DataFrame(
    {
        "tfp": m13_2.summary(round_to='none')['mean'],
        "pystan": [df['sigma'].mean(), df['a_bar'].mean(),
                   df['a.1'].mean(), df['a.2'].mean(),
                   df['a.3'].mean(), df['a.4'].mean(),
                   df['a.5'].mean(), df['a.6'].mean(),
                   df['a.7'].mean(), df['a.8'].mean(),
                   df['a.9'].mean(), df['a.10'].mean(),
                   df['a.11'].mean(), df['a.12'].mean(),
                   df['a.13'].mean(), df['a.14'].mean(),
                   df['a.15'].mean(), df['a.16'].mean(),
                   df['a.17'].mean(), df['a.18'].mean(),
                   df['a.19'].mean(), df['a.20'].mean(),
                   df['a.21'].mean(), df['a.22'].mean(),
                   df['a.23'].mean(), df['a.24'].mean(),
                   df['a.25'].mean(), df['a.26'].mean(),
                   df['a.27'].mean(), df['a.28'].mean(),
                   df['a.29'].mean(), df['a.30'].mean(),
                   df['a.31'].mean(), df['a.32'].mean(),
                   df['a.33'].mean(), df['a.34'].mean(),
                   df['a.35'].mean(), df['a.36'].mean(),
                   df['a.37'].mean(), df['a.38'].mean(),
                   df['a.39'].mean(), df['a.40'].mean(),
                   df['a.41'].mean(), df['a.42'].mean(),
                   df['a.43'].mean(), df['a.44'].mean(),
                   df['a.45'].mean(), df['a.46'].mean(),
                   df['a.47'].mean(), df['a.48'].mean()
                   ]
    })

,tfp,pystan
sigma[0],1.613675,1.620110
a_bar[0],1.341503,1.344551
"alpha[0, 0]",2.105418,2.134119
"alpha[0, 1]",3.056733,3.068841
"alpha[0, 2]",1.010140,0.996831
"alpha[0, 3]",3.070333,3.087345
"alpha[0, 4]",2.105167,2.154708
"alpha[0, 5]",2.119995,2.137342
"alpha[0, 6]",3.079180,3.080906
"alpha[0, 7]",2.161371,2.133229


## 13. Varying effects
### 13.1. Speed comparaison

In [ ]:
from src.main import*
from tensorflow_probability import bijectors as tfb
a = 3.5  # average morning wait time
b = -1  # average difference afternoon wait time
sigma_a = 1  # std dev in intercepts
sigma_b = 0.5  # std dev in slopes
rho = -0.7  # correlation between intercepts and slopes
Mu = tf.constant([a, b])
cov_ab = sigma_a * sigma_b * rho
Sigma = tf.constant([[sigma_a ** 2, cov_ab], [cov_ab, sigma_b ** 2]])
tf.transpose(tf.reshape(tf.constant([1, 2, 3, 4]), (2, 2)))
sigmas = tf.constant([sigma_a, sigma_b])  # standard deviations
Rho = tf.constant([[1, rho], [rho, 1]])  # correlation matrix

# now matrix multiply to get covariance matrix
Sigma = tf.linalg.tensor_diag(sigmas) @ Rho @ tf.linalg.tensor_diag(sigmas)
Sigma
N_cafes = 20

def build_vary_effects():
    _seed = 5
    tf.random.set_seed(_seed)

    seed = tfp.util.SeedStream(_seed, salt="vary_effects")

    Mu = tf.constant([a, b])

    vary_effects = tfd.MultivariateNormalTriL(
        loc=Mu, scale_tril=tf.linalg.cholesky(Sigma)
    ).sample((N_cafes,), seed=seed())

    return vary_effects

vary_effects = build_vary_effects()
a_cafe = vary_effects[:, 0]
b_cafe = vary_effects[:, 1]
N_visits = 10
afternoon = np.tile(np.arange(2), N_visits * N_cafes // 2)
cafe_id = np.repeat(np.arange(N_cafes), N_visits)

def generate_data_frame():
    sigma = 0.5  # std dev within cafes

    _seed = 22
    tf.random.set_seed(_seed)

    seed = tfp.util.SeedStream(_seed, salt="generate_data_frame")

    mu = tf.gather(a_cafe, cafe_id) + tf.gather(b_cafe, cafe_id) * afternoon

    wait = tfd.Normal(loc=mu, scale=sigma).sample(seed=seed())
    d = pd.DataFrame(dict(cafe=cafe_id, afternoon=afternoon, wait=wait))

    return d
d = generate_data_frame()

d.head()


,cafe,afternoon,wait
0,0,0,3.769121
1,0,1,1.247533
2,0,0,3.812889
3,0,1,2.594510
4,0,0,3.115933


In [ ]:
formula = dict(main = 'y ~ Normal(mu, sigma)',
likelihood1 = 'mu ~ a_cafe_b_cafe[cafe] + a_cafe_b_cafe[cafe]*afternoon',
prior0 = 'a_cafe_b_cafe ~ MultivariateNormalTriL(concat([alpha, beta],axis=-1), LinearOperatorDiag(sigma_alpha_beta).matmul(Rho))',
prior1 = 'sigma ~ Exponential(1)',
prior2 = 'sigma_alpha_beta ~ Exponential(1)',
prior3 = 'alpha ~ Normal(5,2)',
prior4 = 'beta ~ Normal(-1,0.5)',
prior5 = 'Rho ~ LKJ(2,2)',
)


start = tm.time()
m14_1 = model(formula, d)
m14_1.fit(observed_data = dict(y = d.wait.astype('float32').values),num_chains=4)
end = tm.time()    
print(f"BI took: {end - start:.4f} seconds")

2024-03-14 10:00:36.000706: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:00:36.000770: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:00:36.000784: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:00:36.000962: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-14 10:00:36.000974: I tensorflow/core/common_runtime/gpu/gpu

BI took: 11.4408 seconds


In [ ]:
import stan
import nest_asyncio
import httpstan.models
import httpstan.cache
try:
  httpstan.cache.delete_model_directory(httpstan.models.calculate_model_name(stan_code)) # Delete  model in cache
except:
  pass

nest_asyncio.apply()
stan_code = """ 
data{
    vector[200] wait;
    array[200] int afternoon;
    array[200] int cafe;
}
parameters{
    vector[20] b_cafe;
    vector[20] a_cafe;
    real a;
    real b;
    vector<lower=0>[2] sigma_cafe;
    real<lower=0> sigma;
    corr_matrix[2] Rho;
}
model{
    vector[200] mu;
    Rho ~ lkj_corr( 2 );
    sigma ~ exponential( 1 );
    sigma_cafe ~ exponential( 1 );
    b ~ normal( -1 , 0.5 );    
    a ~ normal( 5 , 2 );
    {
        array[20] vector[2] YY;
        vector[2] MU;
        MU = [ a , b ]';
        for ( j in 1:20 ) YY[j] = [ a_cafe[j] , b_cafe[j] ]';
        YY ~ multi_normal( MU , quad_form_diag(Rho , sigma_cafe) );
    }
    for ( i in 1:200 ) {
        mu[i] = a_cafe[cafe[i]] + b_cafe[cafe[i]] * afternoon[i];        
    }
    
    wait ~ normal( mu , sigma );

}
"""
data = {
    'wait' : d['wait'].values.astype(float),
    'afternoon' : d['afternoon'].values.astype(int),
    'cafe' : d['cafe'].values.astype(int)+1,
}
start = tm.time()
stan_model = stan.build(stan_code, data = data)
fit = stan_model.sample(num_chains=4, num_samples=2000, num_warmup = 500)
end = tm.time()    
df = fit.to_frame()
print(f"Pystan took: {end - start:.4f} seconds")

Building...



Building: 23.1s, done.Messages from stanc:
Warning in '/tmp/httpstan_ex2_cpo7/model_y5f2lzzr.stan', line 18, column 4: It
    is suggested to reparameterize your model to replace lkj_corr with
    lkj_corr_cholesky, the Cholesky factor variant. lkj_corr tends to run
    slower, consume more memory, and has higher risk of numerical errors.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:  25% (2500/10000)
Sampling:  50% (5000/10000)
Sampling:  75% (7500/10000)
Sampling: 100% (10000/10000)
Sampling: 100% (10000/10000), done.
Messages received during sampling:
  Gradient evaluation took 8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.8 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be reject

Pystan took: 25.4652 seconds


### 13.2. Output comparaison

In [ ]:
pd.DataFrame(
    {
        "tfp": m14_1.summary(round_to='none')['mean'],
        "pystan": [df['sigma_cafe.1'].mean(),
                   df['sigma_cafe.2'].mean(),
                   df['b'].mean(),df['a'].mean(),
                   df['Rho.1.1'].mean(),df['Rho.2.1'].mean(),
                   df['Rho.1.2'].mean(),df['Rho.2.2'].mean(),
                   df['a_cafe.1'].mean(), df['b_cafe.1'].mean(),
                   df['a_cafe.2'].mean(), df['b_cafe.2'].mean(),
                   df['a_cafe.3'].mean(), df['b_cafe.3'].mean(),
                   df['a_cafe.4'].mean(), df['b_cafe.4'].mean(),
                   df['a_cafe.5'].mean(), df['b_cafe.5'].mean(),
                   df['a_cafe.6'].mean(), df['b_cafe.6'].mean(),
                   df['a_cafe.7'].mean(), df['b_cafe.7'].mean(),
                   df['a_cafe.8'].mean(), df['b_cafe.8'].mean(),
                   df['a_cafe.9'].mean(), df['b_cafe.9'].mean(),
                   df['a_cafe.10'].mean(), df['b_cafe.10'].mean(),
                   df['a_cafe.11'].mean(), df['b_cafe.11'].mean(),
                   df['a_cafe.12'].mean(), df['b_cafe.12'].mean(),
                   df['a_cafe.13'].mean(), df['b_cafe.13'].mean(),
                   df['a_cafe.14'].mean(), df['b_cafe.14'].mean(),
                   df['a_cafe.15'].mean(), df['b_cafe.15'].mean(),
                   df['a_cafe.16'].mean(), df['b_cafe.16'].mean(),
                   df['a_cafe.17'].mean(), df['b_cafe.17'].mean(),
                   df['a_cafe.18'].mean(), df['b_cafe.18'].mean(),
                   df['a_cafe.19'].mean(), df['b_cafe.19'].mean(),
                   df['a_cafe.20'].mean(), df['b_cafe.20'].mean(),
                   df['sigma'].mean()]
    })

,tfp,pystan
sigma_alpha_beta[0],1.018171,1.007958
sigma_alpha_beta[1],0.639478,0.637235
beta[0],-1.042332,-1.041170
alpha[0],3.675004,3.672552
"Rho[0, 0]",1.000000,1.000000
"Rho[0, 1]",0.000000,-0.636808
"Rho[1, 0]",-0.667479,-0.636808
"Rho[1, 1]",0.719806,1.000000
"a_cafe_b_cafe[0, 0]",3.027795,3.029698
"a_cafe_b_cafe[0, 1]",-0.966856,-0.974763
